# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [ ]:
# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución

user = 'navia' # navia, roberto
modelo = 'hybrid'

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)

if user == 'navia':
    sc.addPyFile("/export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    sc.addPyFile("/export/usuarios01/navia/spark/SVM_spark/common/lib/quadtree_utils.py")   
    import svm_utils as SVM_UTILS
    import quadtree_utils as QUADTREE
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    %matplotlib inline

kdatasets = [1, 2, 3, 4]
kfold = 1
Niter = 150
#Niter = 2
Nnodes = 32
Samplefraction = 0.05

NCs = [5, 10, 25, 50, 100, 200]

for kdataset in kdatasets:
    aucs = []
    times = []
    x_tr, y_tr, x_tst, y_tst, sigma, C, NC, name_dataset = SVM_UTILS.load_data(kdataset)
    XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
    XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
    for NC in NCs:
        if modelo == 'hybrid':
            auc_tst, elapsed_time = SVM_UTILS.train_hybridSVM(XtrRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
        aucs.append(auc_tst)
        times.append(elapsed_time)
    print aucs
    print times
    filename = 'results_dataset_' + str(kdataset) + '_' + modelo + '.pkl'
    with open(filename, 'w') as f:
        pickle.dump([aucs, times], f)
